In [ ]:
document = """The processes we want to describe are recording the product on the shelf, the customer collecting product, and payment after choosing items; in this part, we have two types of customers: Normal and Extra-customer.

Before purchasing products from different continents, the supermarket puts on store according to or following on type of each product. They use physical inventory where every product purchased is recorded by using the form of stock. When the chief of the store wants to check the number on the store, they take all forms for each product and then count products to verify if the number written on the form matches the number of products stored.

After purchasing a new product or getting the outcome from a supplier, those products are recorded in stock. For selling, Products are recorded on the shelf of supermarkets. Others to the cold chamber (cheese, meat) or in the fridge and freezer sometimes come from the cold chamber toward the freezer before being recorded to the shelf, checking the product's expiration date.

All products are recorded according to the group of each product, the group of alimentation products, the group of pastry/bakery, and butcher products. After recording all products on stock and shelf, it is necessary to record on the cashier's machine where recordable depending on the group. There is a person loaded that action in the supermarket who controller when the shelf is taken care of or occupied of products.

When clients or customers enter the supermarket, saw the product wanted in a different group of products, and some employees can help us or orient clients/customers by showing where products are placed. After choosing and selecting products, one can take a basket for storing goods; before going, check the price, expiration date, and ingredients, then go to the cashier. Arriving there, they deposit around him; the cashier takes one by one entering on machine according also on a group of the product when finished, gives us to the controller to make the product on free packing. When the cashier finishes those transactions, they calculate and show the client/customer on-screen of the machine and tell the customer total of money they can pay, and give us an invoice when the customer or client needed.

The delivery process begins when the customer first interacts with the service organization and ends when the delivery of the desired service is completed and the customer exits the process. At the supermarket, after getting the order from the chief of order, the bakery and pastry agent prepare the goods and quantities of the order. When finished, they package according to each category of product. Then the chief of order checks and counts if there is no mistake; when the number of orders written to the proforma invoice is well prepared, they give the order for put out before arriving to the customer. Some controllers check again to see if there is some product stolen. After those actions, customers get their order of items ordered; after delivery of that product, the stump and original of the proforma invoice are transferred to the office of the general director.

After selecting the product and calculating the total money, the payment to the customer is made in the following ways:
Cash: where the customer presents cash to the cashier or where the cashier receives money, and the customer gets an invoice. 
Cheques: where customers present checks to the cashier for extra-person, not every customer, for example, MINISANTE, CNLS, Rwanda Revenue Authority, and Top Tower Hotel
"""


# Data Preprocessing

In [ ]:
# remove newlines, and extra spaces
import re

document = re.sub(r'\n', ' ', document)
document = re.sub(r'\s+', ' ', document)

document

In [ ]:
import pandas as pd

#separate document by sentence into a df
document_sentence = pd.DataFrame(document.split('.'), columns=['sentence'])
document_sentence

# Categorization of data

In [ ]:
# taking whether this is useful for Class Diagram or not
document_sentence['useful'] = 0

import os
from tqdm import tqdm
import time
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_URL_1", "")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_1", "")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY_1", "")
api_version = os.getenv("AZURE_OPENAI_API_VERSION_1")

print(endpoint)
print(deployment)
print(subscription_key)
print(api_version)

# Initialize Azure OpenAI Service client with key-based authentication    
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version=api_version,
)




In [ ]:
print("Running labelling")
for index, row in tqdm(document_sentence.iterrows(), total=len(document_sentence)):
    chat_prompt = [{
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are an AI Model that help to classify whether a sentence can be used to support information to create Class Diagram or Use Case Diagram or Activity Diagram\n\nBelow are given 5 sentences and whether it is useful for any of the diagram\n1. \"AP : As a case progresses , I need to record all the individuals and organizations that Verdict: take part in the case activities and the specific role they play .\"\nUseful for Class Diagram, Useful for Use Case Diagram, Useful for Activity Diagram\n\n2. \"Unless you are a celebrity or a good friend of Romano you will need a reservation .\"\nVerdict: Useful for Class Diagram, Useful for Use Case Diagram, Not Useful for Activity Diagram\n\n3. \"Therefore , there can be overlapping table reservations .\"\nVerdict: Useful for Class Diagram, Not Useful for Use Case Diagram, Not Useful for Activity Diagram\n\n4. \"These samples are sometimes sub - divided and distributed to multiple research teams or labs for different specialized observations .\"\nVerdict: Not Useful for Class Diagram, Useful for Use Case Diagram, Useful for Activity Diagram\n\n5. \"When the reservation party arrives at Romano 's the reservation is assigned to one waiter .\"\nVerdict: Useful for Class Diagram, Useful for Use Case Diagram, Not Useful for Activity Diagram\n\n6. \"Yes , we do have customers rent two or more vehicles at the same time .\"\nVerdict: Useful for Class Diagram, Useful for Use Case Diagram, Not Useful for Activity Diagram\n\n7. \"Geological samples are retrieved from the field and then processed in the laboratory to determine various properties , including chemistry , mineralogy , age , and petrophysical properties like density , porosity , permeability .\"\nVerdict: Useful for Class Diagram, Not Useful for Use Case Diagram, Useful for Activity Diagram\n\n8. \"For a hygienist 's appointment , preparation could be as simple as seating the patient in dental chair and putting a bib around his or her neck .\"\nVerdict: Not Useful for Class Diagram, Not Useful for Use Case Diagram, Useful for Activity Diagram\n\n9. \"AP : Actually , that is a constant source of confusion and pain .\"\nVerdict: Not Useful for Class Diagram, Not Useful for Use Case Diagram, Not Useful for Activity Diagram\n\n10. \"Romano tends to overbook tables .\"\nVerdict: Not Useful for Class Diagram, Useful for Use Case Diagram, Not Useful for Activity Diagram\n\nUser will put a sentence and decide whether it will be useful for any of the category as an example output like this [Useful Class, Useful Use Case, Not Useful Activity]\n\n\n"
            }
        ]
    }, {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": row['sentence']
            }
        ]
    }]

    if index % 30 == 0 and index != 0:
        time.sleep(60)

    # Generate the completion  
    completion = client.chat.completions.create(
        model=deployment,
        messages=chat_prompt,
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False
    )
    result = completion.choices[0].message.content
    document_sentence.at[index, 'verdict'] = result
    if "Useful Class" in result and "Not Useful Class" not in result:
        document_sentence.at[index, 'useful'] = 1

document_sentence.to_csv("checkpoint_document_sentence_class.csv")

In [ ]:
# read from checkpoint_document_sentence_class.csv
import pandas as pd
document_sentence = pd.read_csv("checkpoint_document_sentence_class.csv")
sentence_class_diagram_only = document_sentence[document_sentence['useful'] == 1]
sentence_class_diagram_only

# Class Diagram Extraction



In [ ]:
# combine all sentences to a single document
document_class = ' '.join(sentence_class_diagram_only['sentence'].tolist())
document_class

In [ ]:
from transformers import pipeline

# Load Hugging Face pipeline
model_name = "../microsoft/deberta-v3-large-4-epoch-8-bs"
ner_pipeline = pipeline("ner", model=model_name, aggregation_strategy="simple")

text = document_class
entities = ner_pipeline(text)
entities

# Visualize


In [ ]:
colors = {
    "CLASS": "#ffff00",
    "ATTR": "#9932cc",
}
options = {"ents": ["CLASS", "ATTR"], "colors": colors}
# Convert Hugging Face output to spaCy format
from spacy import displacy

spacy_ents = {
    "text": text,
    "ents": [
        {"start": ent["start"], "end": ent["end"], "label": ent["entity_group"]}
        for ent in entities
    ],
    "title": "Named Entity Recognition",
}

# Render the visualization
displacy.render(spacy_ents, style="ent", manual=True, jupyter=True, options = options)

# Group Class and Attributes

In [ ]:
from collections import defaultdict

# Given entity list
entities_to_group = entities

# Initialize dictionary
summary = {
    "class": defaultdict(int),
    "attr": defaultdict(int)
}

# Process entities
for ent in entities_to_group:
    entity_type = ent["entity_group"]
    word = ent["word"]
    
    # Store counts based on entity type
    if entity_type in summary:
        summary[entity_type][word] += 1

# Convert defaultdict to normal dict
summary = {key: list(value.keys()) for key, value in summary.items()}

# Format output
summary_string = f"class: {summary['class']}, attribute: {summary['attr']}, description: {document_class}"

print(summary_string)

In [ ]:
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You will be given a JSON of class names, attributes, and a system description. Your task is to generate plantuml script containing classes, attributes, and relationships according to the system description. Strictly produce only plantuml script"
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": summary_string
            }
        ]
    }
]

# Generate the completion  
completion = client.chat.completions.create(
    model=deployment,
    messages=chat_prompt,
    max_tokens=800,
    temperature=0,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
)
result = completion.choices[0].message.content
result

In [ ]:
result = result.strip('```plantuml')
result = result.strip('```')
with open("code.txt", "w", encoding="utf-8") as file:
    file.write(result)